## Biblioteki

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from openpyxl import load_workbook

## Pobór danych 

In [2]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Długość ścieżki

In [3]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Algorytm genetyczny (GA)

In [4]:
def crossover_order(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]

    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = gene

    return child

def crossover_pmx(parent1, parent2):
    size = len(parent1)
    start, end = 2, 6
    child = [None] * size

    # Skopiuj segment z 1-ego rodzica do dziecka
    child[start:end] = parent1[start:end]

    # Wypełnij pozostałe pozycje z 2-ego rodzica
    for i in range(size):
        if child[i] is None:
            gene = parent2[i]
            while gene in child:
                gene = parent2[parent1.index(gene)]
            child[i] = gene

    return child

def selection_tournament(population, scores, k=3):
    # Wybierz k losowych osobników z populacji
    selected = random.sample(range(len(population)), k)
    # Posortuj wybrane osobniki według ich wyników
    selected = sorted(selected, key=lambda x: scores[x])
    # Zwróć najlepszego osobnika
    return population[selected[0]]

def selection_roulette(population, scores):
    total_score = sum(scores)
    
    # Oblicz prawdopodobieństwa
    probabilities = [score / total_score for score in scores]

    # Oblicz skumulowane prawdopodobieństwa
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    
    # Wybierz losową liczbę między 0 a 1
    pick = random.uniform(0, 1)

    # Wybierz osobnika na podstawie losowej liczby
    for i, cumulative_probability in enumerate(cumulative_probabilities):
        if pick <= cumulative_probability:
            return population[i]


def mutate_swap(individual):
    # Zamień dwa losowe geny miejscami
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]

def mutate_inversion(individual):
    # Odwróć kolejność genów między dwoma losowymi punktami
    i, j = sorted(random.sample(range(len(individual)), 2))
    individual[i:j] = reversed(individual[i:j])

def mutate_insertion(individual):
    # Wstaw losowy gen w inne miejsce
    i, j = random.sample(range(len(individual)), 2)
    gene = individual.pop(i)
    individual.insert(j, gene)


def genetic_algorithm(distance_matrix, population_size=10, generations=5000, crossover_rate=0.6, mutation_rate=0.15, selection_method='tournament', crossover_method='order', mutation_method='inversion'):
    start_time = time.time()

    n = len(distance_matrix)
    # Inicjalizuj populację losowymi permutacjami
    population = [random.sample(range(n), n) for _ in range(population_size)]
    best_solution = None
    best_cost = float('inf')
    best_time = 0

    for generation in range(1, generations + 1):
        # Oblicz wyniki dla populacji
        scores = [calculate_path_cost(distance_matrix, individual) for individual in population]
        new_population = []

        for _ in range(population_size // 2):
            # Wybierz rodziców
            if selection_method == 'tournament':
                parent1 = selection_tournament(population, scores)
                parent2 = selection_tournament(population, scores)
            elif selection_method == 'roulette':
                parent1 = selection_roulette(population, scores)
                parent2 = selection_roulette(population, scores)

            # Krzyżowanie rodziców
            if random.random() < crossover_rate:
                if crossover_method == 'order':
                    child1 = crossover_order(parent1, parent2)
                    child2 = crossover_order(parent2, parent1)
                elif crossover_method == 'pmx':
                    child1 = crossover_pmx(parent1, parent2)
                    child2 = crossover_pmx(parent2, parent1)
            else:
                child1, child2 = parent1[:], parent2[:]

            # Mutacja dzieci
            if random.random() < mutation_rate:
                if mutation_method == 'swap':
                    mutate_swap(child1)
                    mutate_swap(child2)
                elif mutation_method == 'inversion':
                    mutate_inversion(child1)
                    mutate_inversion(child2)
                elif mutation_method == 'insertion':
                    mutate_insertion(child1)
                    mutate_insertion(child2)

            new_population.extend([child1, child2])

        population = new_population

        # Znajdź najlepszego osobnika w populacji
        for individual in population:
            cost = calculate_path_cost(distance_matrix, individual)
            if cost < best_cost:
                best_time = time.time() - start_time
                best_solution = individual
                best_cost = cost

        # Wyświetl informacje co 100 iteracji lub na końcu
        if (generation % 100 == 0) or (generation == generations):
            elapsed_time = time.time() - start_time  # Oblicz czas, który upłynął
            print(f"Iteration {generation}, Best Cost: {best_cost: .2f}, Elapsed Time: {elapsed_time:.2f} seconds")

    best_solution = [x + 1 for x in best_solution]  # Zmień indeksy na numery miast
    return best_solution, best_cost, best_time

### Generowanie wyników

Za podstawowe dane przyjmujemy:

* population_size = 10 

* generations = 5000

* crossover_rate = 0.6

* mutation_rate = 0.15

* selection_method = `tournament`
    
* crossover_method = `order`
    
* mutation_method = `swap`

In [5]:
def run_multiple_tests(data, dataset_name, num_tests=10):
    results = []
    for _ in range(num_tests):
        best_path, best_cost, best_time = genetic_algorithm(data)
        results.append({
            "WYNIK": best_cost,
            "ŚCIEŻKA": best_path,
            "CZAS": best_time,
            "DATASET": dataset_name
        })
    return results

In [6]:
all_results = []
all_results.extend(run_multiple_tests(data1, "DATA1"))
all_results.extend(run_multiple_tests(data2, "DATA2"))
all_results.extend(run_multiple_tests(data3, "DATA3"))

m1 = pd.DataFrame(all_results)

with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='w') as writer:
    m1.to_excel(writer, sheet_name = "Basic", index=False)

Iteration 100, Best Cost:  24469.00, Elapsed Time: 0.04 seconds
Iteration 200, Best Cost:  22189.00, Elapsed Time: 0.08 seconds
Iteration 300, Best Cost:  18299.00, Elapsed Time: 0.12 seconds
Iteration 400, Best Cost:  17513.00, Elapsed Time: 0.15 seconds
Iteration 500, Best Cost:  15388.00, Elapsed Time: 0.19 seconds
Iteration 600, Best Cost:  14686.00, Elapsed Time: 0.22 seconds
Iteration 700, Best Cost:  12886.00, Elapsed Time: 0.25 seconds
Iteration 800, Best Cost:  12886.00, Elapsed Time: 0.28 seconds
Iteration 900, Best Cost:  12481.00, Elapsed Time: 0.31 seconds
Iteration 1000, Best Cost:  12481.00, Elapsed Time: 0.35 seconds
Iteration 1100, Best Cost:  12398.00, Elapsed Time: 0.38 seconds
Iteration 1200, Best Cost:  12398.00, Elapsed Time: 0.42 seconds
Iteration 1300, Best Cost:  12054.00, Elapsed Time: 0.45 seconds
Iteration 1400, Best Cost:  11947.00, Elapsed Time: 0.48 seconds
Iteration 1500, Best Cost:  11947.00, Elapsed Time: 0.51 seconds
Iteration 1600, Best Cost:  11839.

#### Badanie wpływu poszczególnych parametrów

Badanie wpływu parametru *population_size*

In [7]:
def test_population_size(data, dataset_name, num_repeats=10):
    results = []
    population_sizes = [5, 10, 15, 20]
    for population_size in population_sizes:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, population_size=population_size)
            results.append({"PARAMETR": population_size, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_population_size_1 = test_population_size(data1, "DATA1")
df_population_size_2 = test_population_size(data2, "DATA2")
df_population_size_3 = test_population_size(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
population_size = pd.concat([df_population_size_1, df_population_size_2, df_population_size_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    population_size.to_excel(writer, sheet_name='population_size', index=False)

Iteration 100, Best Cost:  34874.00, Elapsed Time: 0.03 seconds
Iteration 200, Best Cost:  30079.00, Elapsed Time: 0.04 seconds
Iteration 300, Best Cost:  26011.00, Elapsed Time: 0.06 seconds
Iteration 400, Best Cost:  25042.00, Elapsed Time: 0.07 seconds
Iteration 500, Best Cost:  23972.00, Elapsed Time: 0.08 seconds
Iteration 600, Best Cost:  23226.00, Elapsed Time: 0.09 seconds
Iteration 700, Best Cost:  20503.00, Elapsed Time: 0.11 seconds
Iteration 800, Best Cost:  20503.00, Elapsed Time: 0.13 seconds
Iteration 900, Best Cost:  20064.00, Elapsed Time: 0.14 seconds
Iteration 1000, Best Cost:  19825.00, Elapsed Time: 0.16 seconds
Iteration 1100, Best Cost:  19825.00, Elapsed Time: 0.19 seconds
Iteration 1200, Best Cost:  19825.00, Elapsed Time: 0.25 seconds
Iteration 1300, Best Cost:  19825.00, Elapsed Time: 0.28 seconds
Iteration 1400, Best Cost:  19825.00, Elapsed Time: 0.31 seconds
Iteration 1500, Best Cost:  19825.00, Elapsed Time: 0.32 seconds
Iteration 1600, Best Cost:  19825.

Badanie wpływu parametru *mutation_rate*

In [8]:
def test_mutation_rate(data, dataset_name, num_repeats=10):
    results = []
    mutation_rates = [0.01, 0.05, 0.1, 0.2]
    for mutation_rate in mutation_rates:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, mutation_rate=mutation_rate)
            results.append({"PARAMETR": mutation_rate, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_mutation_rate_1 = test_mutation_rate(data1, "DATA1")
df_mutation_rate_2 = test_mutation_rate(data2, "DATA2")
df_mutation_rate_3 = test_mutation_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
mutation_rate = pd.concat([df_mutation_rate_1, df_mutation_rate_2, df_mutation_rate_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    mutation_rate.to_excel(writer, sheet_name='mutation_rate', index=False)

Iteration 100, Best Cost:  35623.00, Elapsed Time: 0.05 seconds
Iteration 200, Best Cost:  33555.00, Elapsed Time: 0.12 seconds
Iteration 300, Best Cost:  32131.00, Elapsed Time: 0.17 seconds
Iteration 400, Best Cost:  31447.00, Elapsed Time: 0.21 seconds
Iteration 500, Best Cost:  28958.00, Elapsed Time: 0.26 seconds
Iteration 600, Best Cost:  28214.00, Elapsed Time: 0.30 seconds
Iteration 700, Best Cost:  26999.00, Elapsed Time: 0.34 seconds
Iteration 800, Best Cost:  26022.00, Elapsed Time: 0.38 seconds
Iteration 900, Best Cost:  25773.00, Elapsed Time: 0.42 seconds
Iteration 1000, Best Cost:  25773.00, Elapsed Time: 0.47 seconds
Iteration 1100, Best Cost:  25773.00, Elapsed Time: 0.51 seconds
Iteration 1200, Best Cost:  24828.00, Elapsed Time: 0.56 seconds
Iteration 1300, Best Cost:  24507.00, Elapsed Time: 0.61 seconds
Iteration 1400, Best Cost:  24297.00, Elapsed Time: 0.64 seconds
Iteration 1500, Best Cost:  24105.00, Elapsed Time: 0.67 seconds
Iteration 1600, Best Cost:  24105.

Badanie wpływu parametru *crossover_rate*

In [9]:
def test_crossover_rate(data, dataset_name, num_repeats=10):
    results = []
    crossover_rates = [0.6, 0.7, 0.8, 0.9]
    for crossover_rate in crossover_rates:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, crossover_rate=crossover_rate)
            results.append({"PARAMETR": crossover_rate, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_crossover_rate_1 = test_crossover_rate(data1, "DATA1")
df_crossover_rate_2 = test_crossover_rate(data2, "DATA2")
df_crossover_rate_3 = test_crossover_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
crossover_rate = pd.concat([df_crossover_rate_1, df_crossover_rate_2, df_crossover_rate_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    crossover_rate.to_excel(writer, sheet_name='crossover_rate', index=False)

Iteration 100, Best Cost:  26236.00, Elapsed Time: 0.08 seconds
Iteration 200, Best Cost:  22209.00, Elapsed Time: 0.12 seconds
Iteration 300, Best Cost:  21180.00, Elapsed Time: 0.14 seconds
Iteration 400, Best Cost:  19436.00, Elapsed Time: 0.17 seconds
Iteration 500, Best Cost:  15995.00, Elapsed Time: 0.20 seconds
Iteration 600, Best Cost:  15586.00, Elapsed Time: 0.23 seconds
Iteration 700, Best Cost:  14394.00, Elapsed Time: 0.26 seconds
Iteration 800, Best Cost:  13934.00, Elapsed Time: 0.31 seconds
Iteration 900, Best Cost:  13579.00, Elapsed Time: 0.34 seconds
Iteration 1000, Best Cost:  13217.00, Elapsed Time: 0.38 seconds
Iteration 1100, Best Cost:  12455.00, Elapsed Time: 0.42 seconds
Iteration 1200, Best Cost:  12332.00, Elapsed Time: 0.46 seconds
Iteration 1300, Best Cost:  12223.00, Elapsed Time: 0.49 seconds
Iteration 1400, Best Cost:  12168.00, Elapsed Time: 0.52 seconds
Iteration 1500, Best Cost:  12168.00, Elapsed Time: 0.55 seconds
Iteration 1600, Best Cost:  12128.

Badanie wpływu parametru *generations*

In [10]:
def test_generations(data, dataset_name, num_repeats=10):
    results = []
    generations_list = [5000, 7500, 10000, 15000]
    for generations in generations_list:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, generations=generations)
            results.append({"PARAMETR": generations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_generations_1 = test_generations(data1, "DATA1")
df_generations_2 = test_generations(data2, "DATA2")
df_generations_3 = test_generations(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
generations = pd.concat([df_generations_1, df_generations_2, df_generations_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    generations.to_excel(writer, sheet_name='generations', index=False)

Iteration 100, Best Cost:  23379.00, Elapsed Time: 0.06 seconds
Iteration 200, Best Cost:  19043.00, Elapsed Time: 0.09 seconds
Iteration 300, Best Cost:  18041.00, Elapsed Time: 0.12 seconds
Iteration 400, Best Cost:  16729.00, Elapsed Time: 0.15 seconds
Iteration 500, Best Cost:  15543.00, Elapsed Time: 0.18 seconds
Iteration 600, Best Cost:  14708.00, Elapsed Time: 0.21 seconds
Iteration 700, Best Cost:  14229.00, Elapsed Time: 0.25 seconds
Iteration 800, Best Cost:  12336.00, Elapsed Time: 0.27 seconds
Iteration 900, Best Cost:  12313.00, Elapsed Time: 0.30 seconds
Iteration 1000, Best Cost:  12301.00, Elapsed Time: 0.33 seconds
Iteration 1100, Best Cost:  11893.00, Elapsed Time: 0.37 seconds
Iteration 1200, Best Cost:  11893.00, Elapsed Time: 0.40 seconds
Iteration 1300, Best Cost:  11811.00, Elapsed Time: 0.48 seconds
Iteration 1400, Best Cost:  11803.00, Elapsed Time: 0.52 seconds
Iteration 1500, Best Cost:  11737.00, Elapsed Time: 0.56 seconds
Iteration 1600, Best Cost:  11600.

Badanie wpływu parametru *selection_method*

In [11]:
def test_selection_method(data, dataset_name, num_repeats=10):
    results = []
    selection_methods = ['roulette', 'tournament']
    for selection_method in selection_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, selection_method=selection_method)
            results.append({"PARAMETR": selection_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_selection_method_1 = test_selection_method(data1, "DATA1")
df_selection_method_2 = test_selection_method(data2, "DATA2")
df_selection_method_3 = test_selection_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
selection_method = pd.concat([df_selection_method_1, df_selection_method_2, df_selection_method_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    selection_method.to_excel(writer, sheet_name='selection_method', index=False)

Iteration 100, Best Cost:  45672.00, Elapsed Time: 0.06 seconds
Iteration 200, Best Cost:  45672.00, Elapsed Time: 0.09 seconds
Iteration 300, Best Cost:  45672.00, Elapsed Time: 0.12 seconds
Iteration 400, Best Cost:  45672.00, Elapsed Time: 0.15 seconds
Iteration 500, Best Cost:  42070.00, Elapsed Time: 0.18 seconds
Iteration 600, Best Cost:  40923.00, Elapsed Time: 0.23 seconds
Iteration 700, Best Cost:  39894.00, Elapsed Time: 0.26 seconds
Iteration 800, Best Cost:  39894.00, Elapsed Time: 0.29 seconds
Iteration 900, Best Cost:  39894.00, Elapsed Time: 0.33 seconds
Iteration 1000, Best Cost:  39894.00, Elapsed Time: 0.37 seconds
Iteration 1100, Best Cost:  39894.00, Elapsed Time: 0.40 seconds
Iteration 1200, Best Cost:  39894.00, Elapsed Time: 0.43 seconds
Iteration 1300, Best Cost:  39894.00, Elapsed Time: 0.47 seconds
Iteration 1400, Best Cost:  39894.00, Elapsed Time: 0.53 seconds
Iteration 1500, Best Cost:  39894.00, Elapsed Time: 0.56 seconds
Iteration 1600, Best Cost:  39894.

Badanie wpływu parametru *crossover_method*

In [12]:
def test_crossover_method(data, dataset_name, num_repeats=10):
    results = []
    crossover_methods = ['order', 'pmx']
    for crossover_method in crossover_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, crossover_method=crossover_method)
            results.append({"PARAMETR": crossover_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_crossover_method_1 = test_crossover_method(data1, "DATA1")
df_crossover_method_2 = test_crossover_method(data2, "DATA2")
df_crossover_method_3 = test_crossover_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
crossover_method = pd.concat([df_crossover_method_1, df_crossover_method_2, df_crossover_method_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    crossover_method.to_excel(writer, sheet_name='crossover_method', index=False)

Iteration 100, Best Cost:  24726.00, Elapsed Time: 0.08 seconds
Iteration 200, Best Cost:  21793.00, Elapsed Time: 0.11 seconds
Iteration 300, Best Cost:  19698.00, Elapsed Time: 0.14 seconds
Iteration 400, Best Cost:  18416.00, Elapsed Time: 0.17 seconds
Iteration 500, Best Cost:  17056.00, Elapsed Time: 0.20 seconds
Iteration 600, Best Cost:  14972.00, Elapsed Time: 0.23 seconds
Iteration 700, Best Cost:  14597.00, Elapsed Time: 0.25 seconds
Iteration 800, Best Cost:  14397.00, Elapsed Time: 0.28 seconds
Iteration 900, Best Cost:  13999.00, Elapsed Time: 0.31 seconds
Iteration 1000, Best Cost:  13836.00, Elapsed Time: 0.35 seconds
Iteration 1100, Best Cost:  13640.00, Elapsed Time: 0.39 seconds
Iteration 1200, Best Cost:  13640.00, Elapsed Time: 0.45 seconds
Iteration 1300, Best Cost:  12920.00, Elapsed Time: 0.49 seconds
Iteration 1400, Best Cost:  12913.00, Elapsed Time: 0.52 seconds
Iteration 1500, Best Cost:  12902.00, Elapsed Time: 0.55 seconds
Iteration 1600, Best Cost:  12534.

Badanie wpływu parametru *mutation_method*

In [13]:
def test_mutation_method(data, dataset_name, num_repeats=10):
    results = []
    mutation_methods = ['insertion', 'swap', 'inversion']
    for mutation_method in mutation_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, mutation_method=mutation_method)
            results.append({"PARAMETR": mutation_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_mutation_method_1 = test_mutation_method(data1, "DATA1")
df_mutation_method_2 = test_mutation_method(data2, "DATA2")
df_mutation_method_3 = test_mutation_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
mutation_method = pd.concat([df_mutation_method_1, df_mutation_method_2, df_mutation_method_3], ignore_index=True)
with pd.ExcelWriter('GA.xlsx', engine='openpyxl', mode='a') as writer:
    mutation_method.to_excel(writer, sheet_name='mutation_method', index=False)

Iteration 100, Best Cost:  25372.00, Elapsed Time: 0.06 seconds
Iteration 200, Best Cost:  20886.00, Elapsed Time: 0.10 seconds
Iteration 300, Best Cost:  19074.00, Elapsed Time: 0.13 seconds
Iteration 400, Best Cost:  17260.00, Elapsed Time: 0.16 seconds
Iteration 500, Best Cost:  16467.00, Elapsed Time: 0.19 seconds
Iteration 600, Best Cost:  15585.00, Elapsed Time: 0.21 seconds
Iteration 700, Best Cost:  15167.00, Elapsed Time: 0.24 seconds
Iteration 800, Best Cost:  14605.00, Elapsed Time: 0.27 seconds
Iteration 900, Best Cost:  14603.00, Elapsed Time: 0.30 seconds
Iteration 1000, Best Cost:  14454.00, Elapsed Time: 0.35 seconds
Iteration 1100, Best Cost:  14224.00, Elapsed Time: 0.39 seconds
Iteration 1200, Best Cost:  14044.00, Elapsed Time: 0.44 seconds
Iteration 1300, Best Cost:  14044.00, Elapsed Time: 0.49 seconds
Iteration 1400, Best Cost:  13538.00, Elapsed Time: 0.53 seconds
Iteration 1500, Best Cost:  13407.00, Elapsed Time: 0.56 seconds
Iteration 1600, Best Cost:  13321.